In [ ]:
from pymongo import MongoClient
import base64
from sshtunnel import SSHTunnelForwarder
from binascii import hexlify, unhexlify
from symbolchain.facade.SymbolFacade import SymbolFacade
from symbolchain.CryptoTypes import PublicKey
import requests
from tqdm import tqdm
from collections import defaultdict


In [ ]:
MONGO_HOST = "symbolblog-testnet.com" # Update with your node details

server = SSHTunnelForwarder(
    (MONGO_HOST,22),
    ssh_username='', # Add username of node
    ssh_password='', # Add password of node
    remote_bind_address=('172.20.0.2', 27017)
  )
server.start()

In [ ]:
client = MongoClient('localhost', server.local_bind_port)
print(server.local_bind_port)
print("connected")

In [ ]:
# Read in list of node addresses

nodes = []  # Initialize an empty list to store nodes
try:
    with open("list.txt", 'r') as file:
        for line in file:
            nodes.append(line.strip())
except FileNotFoundError:
    print(f"File not found: {list.txt}")


In [ ]:
# Function to fetch JSON data for unlocked accounts (harvesters on the node)
def fetch_json_data(node, timeout=10):  # timeout of 10 seconds
    url = f"http://{node}:3000/node/unlockedaccount"
    try:
        response = requests.get(url, timeout=timeout)
        response.raise_for_status()
        data = response.json()
        return data
    except requests.Timeout:
        print(f"Request to {url} timed out.")
        return None
    except requests.RequestException as e:
        print(f"Request to {url} failed: {e}")
        return None

# Create a dictionary to store nodes and harvesters
node_details = {}

# Loop through each node, fetch JSON data, and add it to the dictionary
for node in tqdm(nodes, desc="Fetching JSON Data"):  # tqdm wrapper for progress bar
    json_data = fetch_json_data(node)
    if json_data and 'unlockedAccount' in json_data:
        node_details[node] = json_data['unlockedAccount']



In [ ]:
facade = SymbolFacade('mainnet')

db = client['catapult']
collection = db.transactions
mapping = defaultdict(dict)

out = collection.find({'transaction.type': 16716})
for x in out:
    if x['transaction']['linkAction'] == 1:
        main = str(facade.network.public_key_to_address(PublicKey((hexlify(x['transaction']['signerPublicKey']).decode('utf8')))))    
        link = str(facade.network.public_key_to_address(PublicKey((hexlify(x['transaction']['linkedPublicKey']).decode('utf8')))))
        mapping[link] = main

address_details = defaultdict(list)
for node, public_keys in node_details.items():
    for pk in public_keys:
        address = str(facade.network.public_key_to_address(PublicKey(pk)))
        if address in mapping:  # Check if the address is in mapping
            address_details[node].append(mapping[address])
        else:
            address_details[node].append(address)


In [ ]:
db = client['catapult']
collection = db.accounts
facade = SymbolFacade('mainnet')

out = collection.find()

balances = {}

for x in out:
    xym = 0
    address = str(base64.b32encode(x['account']['address']).decode('utf8')[0:39])
    mosaics = x['account']['mosaics']
    for mos in mosaics:
        if (mos['id'] == 7777031834025731064):
            balances[address] = mos['amount']/1000000
            xym = 1
    if xym == 0:
        balances[address] = 0


In [ ]:
balance_details = defaultdict(list)

for node, addresses in address_details.items():
    for address in addresses:
        try:
            balance_details[node].append(balances[address])
        except:
            balance_details[node].append(0)

In [ ]:
with open('nodes_balances.tsv', 'w') as file:
    file.write("Node\tHarvesters\tMain\tTotal\n")
    
    # Iterate through the dictionary and write the required information
    for node, balances in balance_details.items():
        list_size = len(balances)
        main = balances[0] if balances else 'N/A'  # Handle empty list case
        total = sum(balances)
        file.write(f"{node}\t{list_size}\t{main}\t{total}\n")
